# Practica Final de ML1.

Usamos un dataset sacado de https://archive.ics.uci.edu/dataset/203/yearpredictionmsd,  con aprox medio millon de instances y noventa atributos donde 12 son sobre el timbre medio y los otros 78 sobre la desviacion estandar del timbre. Nuestro objetivo es, con todos estos datos, determinar el año de la canción  (+ info sobre el dataset y como poder entrenarlo aqui https://samyzaf.com/ML/song_year/song_year.html). Cargamos el dataset, primero como un DataFrame para ver la distribución de los datos y a continuación como array. 

*** No es necesario en ppio guardar los modelos, guardar simplemente los outputs de los modelos***

In [1]:
using Downloads;
using DelimitedFiles;
using CSV;
using DataFrames;
using StatsBase;

data = CSV.File("YearPredictionMSD.txt") |> DataFrame;
describe(data)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Real,Float64,Real,Int64,DataType
1,2001,1998.4,1922,2002.0,2011,0,Int64
2,49.94357,43.3871,1.749,44.2585,61.9701,0,Float64
3,21.47114,1.28952,-337.092,8.41773,384.066,0,Float64
4,73.07750,8.65822,-301.005,10.4762,322.851,0,Float64
5,8.74861,1.16411,-154.184,-0.652855,335.772,0,Float64
6,-17.40628,-6.55358,-181.953,-6.00777,262.069,0,Float64
7,-13.09905,-9.52197,-81.7943,-11.1884,166.237,0,Float64
8,-25.01202,-2.39105,-188.214,-2.04663,172.403,0,Float64
9,-12.23257,-1.79322,-72.5038,-1.73642,126.741,0,Float64


In [2]:
data_or = readdlm("YearPredictionMSD.txt", ',');
# data = filter(year -> year > 1965, data[:,1]);
# data = filter(year -> year < 2010, data[:,1]);
targets = data_or[:,1];
targets = round.(Int,targets);
index = ((targets .> 1995) .& (targets .< 2011))
data = data_or[index,:];
print(size(data))

In [3]:
unique_years = unique(data[:,1]);
new_data = [];
for year in unique_years
    year_data = data[(data[:,1] .== year), :];
    index = collect(1:size(year_data,1));
    index_reduced = StatsBase.sample(1:size(year_data,1), 5000, replace=false)
    year_data = data[index_reduced, :];
    new_data = vcat(new_data, year_data)
end
print(size(new_data))

(371763, 91)

(75000, 91)

Una vez definido en un array los datos para nuestro modelo, deberemos determinar como intentaremos resolver nuestro problema, determinar el año de la canción. Para resolver este problema deberemos tener cuatro formas de tratar los datos diferentes, en nuestro caso usaremos:

1. El dataset entero (modificando el validation set etc).
2. Reducir el dataset con alguno de los metodos de la unit 8. (PCA media 0 y std 1).
3. Hacer agrupaciones nosotros individuales (usamos los datos de media y de std individualmente).

En principio deberíamos realizar la normalización de todos los datos (si nos sobra tiempo podemos probar a no usar la normalización).

In [4]:
include("functions_finalproject.jl");

#CAMBIAR EL ACCURACY PARA CALCULARLO BIEN

trainClassEnsemble (generic function with 1 method)

## Modelo con el dataset entero

Cargamos el dataset entero y definimos los output como la primera columan y los inputr el resto. Debido a la variabilidad de los datos normalizamos los datos con media 0 y desviación estandar 1. Para hacer esto necesitamos cargar las funciones desarrolladas en los anteriores temas. (Tener en cuenta que al tener tantos años diferentes el dataset no tiene muchas canciones para cada año, importante para el crossvalidation, igual en algun fold no tenemos representates de todos los años, hay que quitar el assert de la funcion)

In [5]:
targets = new_data[:,1];
inputs = Matrix{Float32}(new_data[:,2:91]);

inputs = convert(Array{Float32, 2},normalizeZeroMean!(inputs));
@assert isa(inputs, Array{<:Float32,2})

# output_data = oneHotEncoding(output_data);
# @assert isa(output_data, AbstractArray{Bool,2})

#Using random seed so we can repeat the results
Random.seed!(123);

#We divide the model in 10 subsets
k = 10
index = crossvalidation(targets, k);

### ANN

Probamos primero una ANN con una capa oculta simplemente para probar las diferentes transferFunctions (tanh, hardtanh, celu, elu, gelu, hardsigmoid, hardswish, relu, leakyrelu... https://fluxml.ai/Flux.jl/stable/models/activation/#man-activations , https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/ https://mlnotebook.github.io/post/transfer-functions/) en principio tomaremos relu, sigmoid y tanh (más usadas, buscar hardtanh pq es mas barata computacionalmente) y comparamos el resultado. Despues aumentamos el tamaño de las ANN (15,30,90) para esa transfer function y aplicamos dos capas ocultas. 

In [9]:
#Cargar un modelo:
folder_path = "modelann [15], [NNlib.softmax]"

# File path to load the model from inside the folder
model_file_path = joinpath(folder_path, "annmodel10.jld2")

ann_model = nothing


# Load the model from the JLD file
loaded_model = jldopen(model_file_path) do file
     global ann_model
     ann_model = read(file, "model")
end


ann_model(inputs')

In [6]:
# MODELO 1 (SI SE EJECUTA MAS DE UNA VEZ SE SOBREESCRIBE EL CODIGO)
Random.seed!(123);

parameters_ann = Dict("topology" => [15], "transferFunctions" => fill(softmax, 1), "learningRate" => 0.01, "repetitionsTraining" => 1, "validationRatio" => 0.0, "maxEpochsVal" => 20);

modelos = modelCrossValidation(:ANN, parameters_ann, inputs, targets, index);
#8 minutos

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.850375: validation loss : 2.8389282: test loss :2.8525362


Epoch 454: train loss: 2.4457738: validation loss :2.4985514: test loss :2.508011


 
Metrics: 
Weighted sensivity: 0.16227018385291767
Weighted specificity: 0.921836473362547
Weighted precision: 0.150270797972149
Weighted F-score: 0.14647673842453565
Accuracy: 0.16227018385291767
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.728574: validation loss : 2.73129: test loss :2.7256362


...


Epoch 524: train loss: 2.4457064: validation loss :2.50071: test loss :2.5073497
 
Metrics: 
Weighted sensivity: 0.16604427847425982
Weighted specificity: 0.9214812153432581
Weighted precision: 0.16189996736802462
Weighted F-score: 0.15350403423991058
Accuracy: 0.16604427847425982
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 3.0034876: validation loss : 2.9988809: test loss :3.0018253


Epoch 457: train loss: 2.4502368: validation loss :2.5043194: test loss :2.5113842
 
Metrics: 
Weighted sensivity: 0.15877251501000667
Weighted specificity: 0.9210370311916647
Weighted precision: 0.15011922691046423
Weighted F-score: 0.14573913153780763
Accuracy: 0.15877251501000667
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.8087926: validation loss : 2.8031049: test loss :2.8099847


Epoch 1000: train loss: 2.4295409: validation loss :2.4979007: test loss :2.5070488
 
Metrics: 
Weighted sensivity: 0.17002535699986654
Weighted specificity: 0.9215269028438408
Weighted precision: 0.1601343730228092
Weighted F-score: 0.151721200547509
Accuracy: 0.17002535699986654
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.849176: validation loss : 2.8468428: test loss :2.849953


Epoch 1000: train loss: 2.426623: validation loss :2.4943767: test loss :2.5058749
 
Metrics: 
Weighted sensivity: 0.16295208861604163
Weighted specificity: 0.9211946786261279
Weighted precision: 0.1565836563586285
Weighted F-score: 0.14755900543677056
Accuracy: 0.16295208861604163


In [7]:
## MODELO 2
Random.seed!(123);

parameters_ann = Dict("topology" => [15], "transferFunctions" => fill(relu, 1), "learningRate" => 0.01, "repetitionsTraining" => 1, "validationRatio" => 0.0, "maxEpochsVal" => 20);

modelCrossValidation(:ANN, parameters_ann, inputs, targets, index)
#8 minutos

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.850375: validation loss : 2.8389282: test loss :2.8525362


Epoch 454: train loss: 2.4457738: validation loss :2.4985514: test loss :2.508011
 
Metrics: 
Weighted sensivity: 0.16227018385291767
Weighted specificity: 0.921836473362547
Weighted precision: 0.150270797972149
Weighted F-score: 0.14647673842453565
Accuracy: 0.16227018385291767
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.728574: validation loss : 2.73129: test loss :2.7256362


Epoch 419: train loss: 2.4437618: validation loss :2.4894743: test loss :2.5104327
 
Metrics: 
Weighted sensivity: 0.1636242504996669
Weighted specificity: 0.9226025279032005
Weighted precision: 0.1565714008940176
Weighted F-score: 0.15028901985569432
Accuracy: 0.1636242504996669
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.8095796: validation loss : 2.8091407: test loss :2.808501


...

Epoch 432: train loss: 2.4532003: validation loss :2.5040393: test loss :2.495515
 
Metrics: 
Weighted sensivity: 0.1650886784904654
Weighted specificity: 0.9225831729741499
Weighted precision: 0.15697199000697037
Weighted F-score: 0.1548163314375312
Accuracy: 0.1650886784904654
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.9361806: validation loss : 2.930007: test loss :2.9375885


Epoch 524: train loss: 2.4457064: validation loss :2.50071: test loss :2.5073497
 
Metrics: 
Weighted sensivity: 0.16604427847425982
Weighted specificity: 0.9214812153432581
Weighted precision: 0.16189996736802462
Weighted F-score: 0.15350403423991058
Accuracy: 0.16604427847425982
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 3.0034876: validation loss : 2.9988809: test loss :3.0018253


Epoch 457: train loss: 2.4502368: validation loss :2.5043194: test loss :2.5113842
 
Metrics: 
Weighted sensivity: 0.15877251501000667
Weighted specificity: 0.9210370311916647
Weighted precision: 0.15011922691046423
Weighted F-score: 0.14573913153780763
Accuracy: 0.15877251501000667
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.8087926: validation loss : 2.8031049: test loss :2.8099847


Epoch 1000: train loss: 2.4295409: validation loss :2.4979007: test loss :2.5070488
 
Metrics: 
Weighted sensivity: 0.17002535699986654
Weighted specificity: 0.9215269028438408
Weighted precision: 0.1601343730228092
Weighted F-score: 0.151721200547509
Accuracy: 0.17002535699986654
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.849176: validation loss : 2.8468428: test loss :2.849953


Epoch 1000: train loss: 2.426623: validation loss :2.4943767: test loss :2.5058749
 
Metrics: 
Weighted sensivity: 0.16295208861604163
Weighted specificity: 0.9211946786261279
Weighted precision: 0.1565836563586285
Weighted F-score: 0.14755900543677056
Accuracy: 0.16295208861604163


In [8]:
## MODELO 3
Random.seed!(123);

parameters_ann = Dict("topology" => [15], "transferFunctions" => fill(tanh, 1), "learningRate" => 0.01, "repetitionsTraining" => 1, "validationRatio" => 0.0, "maxEpochsVal" => 20);

modelCrossValidation(:ANN, parameters_ann, inputs, targets, index)
#8 minutos

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.850375: validation loss : 2.8389282: test loss :2.8525362


Epoch 454: train loss: 2.4457738: validation loss :2.4985514: test loss :2.508011
 
Metrics: 
Weighted sensivity: 0.16227018385291767
Weighted specificity: 0.921836473362547
Weighted precision: 0.150270797972149
Weighted F-score: 0.14647673842453565
Accuracy: 0.16227018385291767
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.728574: validation loss : 2.73129: test loss :2.7256362


Epoch 419: train loss: 2.4437618: validation loss :2.4894743: test loss :2.5104327
 
Metrics: 
Weighted sensivity: 0.1636242504996669
Weighted specificity: 0.9226025279032005
Weighted precision: 0.1565714008940176
Weighted F-score: 0.15028901985569432
Accuracy: 0.1636242504996669
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.8095796: validation loss : 2.8091407: test loss :2.808501


...


Epoch 457: train loss: 2.4502368: validation loss :2.5043194: test loss :2.5113842
 
Metrics: 
Weighted sensivity: 0.15877251501000667
Weighted specificity: 0.9210370311916647
Weighted precision: 0.15011922691046423
Weighted F-score: 0.14573913153780763
Accuracy: 0.15877251501000667


 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.8087926: validation loss : 2.8031049: test loss :2.8099847


Epoch 1000: train loss: 2.4295409: validation loss :2.4979007: test loss :2.5070488
 
Metrics: 
Weighted sensivity: 0.17002535699986654
Weighted specificity: 0.9215269028438408
Weighted precision: 0.1601343730228092
Weighted F-score: 0.151721200547509
Accuracy: 0.17002535699986654


 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.849176: validation loss : 2.8468428: test loss :2.849953


Epoch 1000: train loss: 2.426623: validation loss :2.4943767: test loss :2.5058749
 
Metrics: 
Weighted sensivity: 0.16295208861604163
Weighted specificity: 0.9211946786261279
Weighted precision: 0.1565836563586285
Weighted F-score: 0.14755900543677056
Accuracy: 0.16295208861604163


In [9]:
## MODELO 4
Random.seed!(123);

parameters_ann = Dict("topology" => [15], "transferFunctions" => fill(sigmoid, 1), "learningRate" => 0.01, "repetitionsTraining" => 1, "validationRatio" => 0.0, "maxEpochsVal" => 20);

modelCrossValidation(:ANN, parameters_ann, inputs, targets, index)
#8 minutos

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.850375: validation loss : 2.8389282: test loss :2.8525362


Epoch 454: train loss: 2.4457738: validation loss :2.4985514: test loss :2.508011
 
Metrics: 
Weighted sensivity: 0.16227018385291767
Weighted specificity: 0.921836473362547
Weighted precision: 0.150270797972149
Weighted F-score: 0.14647673842453565
Accuracy: 0.16227018385291767
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.728574: validation loss : 2.73129: test loss :2.7256362


Epoch 419: train loss: 2.4437618: validation loss :2.4894743: test loss :2.5104327
 
Metrics: 
Weighted sensivity: 0.1636242504996669
Weighted specificity: 0.9226025279032005
Weighted precision: 0.1565714008940176
Weighted F-score: 0.15028901985569432
Accuracy: 0.1636242504996669
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.8095796: validation loss : 2.8091407: test loss :2.808501


Epoch 890: train loss: 2.434398: validation loss :2.4954414: test loss :2.4848316
 
Metrics: 
Weighted sensivity: 0.1704197201865423
Weighted specificity: 0.9226545540457529
Weighted precision: 0.1608347625795416
Weighted F-score: 0.1535260250587919
Accuracy: 0.1704197201865423
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.915867: validation loss : 2.911205: test loss :2.9158163


...

Epoch 524: train loss: 2.4457064: validation loss :2.50071: test loss :2.5073497
 
Metrics: 
Weighted sensivity: 0.16604427847425982
Weighted specificity: 0.9214812153432581
Weighted precision: 0.16189996736802462
Weighted F-score: 0.15350403423991058
Accuracy: 0.16604427847425982
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 3.0034876: validation loss : 2.9988809: test loss :3.0018253


Epoch 457: train loss: 2.4502368: validation loss :2.5043194: test loss :2.5113842
 
Metrics: 
Weighted sensivity: 0.15877251501000667
Weighted specificity: 0.9210370311916647
Weighted precision: 0.15011922691046423
Weighted F-score: 0.14573913153780763
Accuracy: 0.15877251501000667
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.8087926: validation loss : 2.8031049: test loss :2.8099847


Epoch 1000: train loss: 2.4295409: validation loss :2.4979007: test loss :2.5070488
 
Metrics: 
Weighted sensivity: 0.17002535699986654
Weighted specificity: 0.9215269028438408
Weighted precision: 0.1601343730228092
Weighted F-score: 0.151721200547509
Accuracy: 0.17002535699986654
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.849176: validation loss : 2.8468428: test loss :2.849953


Epoch 1000: train loss: 2.426623: validation loss :2.4943767: test loss :2.5058749
 
Metrics: 
Weighted sensivity: 0.16295208861604163
Weighted specificity: 0.9211946786261279
Weighted precision: 0.1565836563586285
Weighted F-score: 0.14755900543677056
Accuracy: 0.16295208861604163


In [10]:
## MODELO 5
Random.seed!(123);

parameters_ann = Dict("topology" => [30], "transferFunctions" => fill(relu, 1), "learningRate" => 0.01, "repetitionsTraining" => 1, "validationRatio" => 0.0, "maxEpochsVal" => 20);

modelCrossValidation(:ANN, parameters_ann, inputs, targets, index)
#7 minutos


 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.8068838: validation loss : 2.8229563: test loss :2.8126187


Epoch 432: train loss: 2.3276868: validation loss :2.4462266: test loss :2.456886
 
Metrics: 
Weighted sensivity: 0.1995736743938183
Weighted specificity: 0.9268892549967472
Weighted precision: 0.1862619755777478
Weighted F-score: 0.18496150220008223
Accuracy: 0.1995736743938183
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.822727: validation loss : 2.8107886: test loss :2.8236678


Epoch 463: train loss: 2.3335197: validation loss :2.4596565: test loss :2.4640214
 
Metrics: 
Weighted sensivity: 0.19840106595602933
Weighted specificity: 0.9251232409459553
Weighted precision: 0.1938592407153051
Weighted F-score: 0.1862112299973186
Accuracy: 0.19840106595602933
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.7315261: validation loss : 2.735244: test loss :2.734073


Epoch 367: train loss: 2.3353646: validation loss :2.4634008: test loss :2.445925
 
Metrics: 
Weighted sensivity: 0.19467021985343105
Weighted specificity: 0.9275960864520424
Weighted precision: 0.19114382160180624
Weighted F-score: 0.18487606453644237
Accuracy: 0.19467021985343105


...


Epoch 634: train loss: 2.316049: validation loss :2.4552128: test loss :2.4496517
 
Metrics: 
Weighted sensivity: 0.20552147239263804
Weighted specificity: 0.9273044739872086
Weighted precision: 0.1962508251654016
Weighted F-score: 0.19349588643140017
Accuracy: 0.20552147239263804
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.896773: validation loss : 2.8898728: test loss :2.8931909


Epoch 1000: train loss: 2.3014977: validation loss :2.4452596: test loss :2.4522355
 
Metrics: 
Weighted sensivity: 0.2068045363575717
Weighted specificity: 0.9284770749134182
Weighted precision: 0.19990524559747522
Weighted F-score: 0.19708777929711638
Accuracy: 0.2068045363575717
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.9303396: validation loss : 2.933392: test loss :2.9372962

Epoch 358: train loss: 2.3513465: validation loss :2.458956: test loss :2.4470034
 
Metrics: 
Weighted sensivity: 0.19257974109168557
Weighted specificity: 0.9267429499284277
Weighted precision: 0.1850298422124491
Weighted F-score: 0.18405921478267626
Accuracy: 0.19257974109168557
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------


------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.7699654: validation loss : 2.77109: test loss :2.7681425


Epoch 578: train loss: 2.31826: validation loss :2.4394908: test loss :2.453879
 
Metrics: 
Weighted sensivity: 0.19671693580675298
Weighted specificity: 0.9267736500174915
Weighted precision: 0.19400808034104852
Weighted F-score: 0.18799165212380575
Accuracy: 0.19671693580675298


In [11]:
## MODELO 6
Random.seed!(123);

parameters_ann = Dict("topology" => [120], "transferFunctions" => fill(relu, 1), "learningRate" => 0.01, "repetitionsTraining" => 1, "validationRatio" => 0.0, "maxEpochsVal" => 20);

modelCrossValidation(:ANN, parameters_ann, inputs, targets, index)
#12 minutos


 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 3.0677173

: validation loss : 3.0767245: test loss :3.074557


Epoch 466: train loss: 1.7469739: validation loss :2.1710966: test loss :2.1728098
 
Metrics: 
Weighted sensivity: 0.35145217159605646
Weighted specificity: 0.9443993699887471
Weighted precision: 0.3497140899389465
Weighted F-score: 0.3487935932971468
Accuracy: 0.35145217159605646
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.9463475: validation loss : 2.9428372: test loss :2.942505


Epoch 523: train loss: 1.70465: validation loss :2.1672843: test loss :2.1833575
 
Metrics: 
Weighted sensivity: 0.365489673550966
Weighted specificity: 0.9454052959771184
Weighted precision: 0.36467236787970186
Weighted F-score: 0.3611842777412837
Accuracy: 0.365489673550966

...

Epoch 0: train loss : 2.9166238: validation loss : 2.9169614: test loss :2.9166658


Epoch 491: train loss: 1.7104533: validation loss :2.187741: test loss :2.1726105
 
Metrics: 
Weighted sensivity: 0.35820098758841584
Weighted specificity: 0.9446255521518958
Weighted precision: 0.35598105648783607
Weighted F-score: 0.3537126709979366
Accuracy: 

0.35820098758841584
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.8640015: validation loss : 2.8633044: test loss :2.8648427


Epoch 343: train loss: 1.7946643: validation loss :2.1898687: test loss :2.2030087
 
Metrics: 
Weighted sensivity: 0.331375950887495
Weighted specificity: 0.941350494084029
Weighted precision: 0.3265363961721681
Weighted F-score: 0.3240425205248778
Accuracy: 0.331375950887495


In [12]:
#MODELO 7
Random.seed!(123);

parameters_ann = Dict("topology" => [240], "transferFunctions" => fill(relu, 1), "learningRate" => 0.01, "repetitionsTraining" => 1, "validationRatio" => 0.0, "maxEpochsVal" => 20);

modelCrossValidation(:ANN, parameters_ann, inputs, targets, index)
#30 minutos


 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 3.157495: validation loss : 3.1632285: test loss :3.1598413


Epoch 649: train loss: 0.9822395: validation loss :1.7904763: test loss :1.7910179
 
Metrics: 
Weighted sensivity: 0.5664801492139622
Weighted specificity: 0.9635768446916251
Weighted precision: 0.5648399741040199
Weighted F-score: 0.5647730523288483
Accuracy: 0.5664801492139622
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.9487572: validation loss : 

2.9502573: test loss :2.9475217
Epoch 593: train loss: 1.0198164: validation loss :1.7883668: test loss :1.8173972
 
Metrics: 
Weighted sensivity: 0.5499000666222519
Weighted specificity: 0.9617088397742677
Weighted precision: 0.5487443230390463
Weighted F-score: 0.5478600226634704
Accuracy: 0.5499000666222519


...

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 3.0264282: validation loss : 3.0261998: test loss :3.0276482


Epoch 593: train loss: 1.0469996: validation loss :1.7969826: test loss :1.8120342
 
Metrics: 
Weighted sensivity: 0.5365007340184171
Weighted specificity: 0.9602303413329173
Weighted precision: 0.5363876973856277
Weighted F-score: 0.5350463027686849
Accuracy: 0.5365007340184171
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 3.0002615: validation loss : 2.9882112: test loss :2.9972959


Epoch 611: train loss: 1.0125903: validation loss :1.7941502: test loss :1.8380762
 
Metrics: 
Weighted sensivity: 0.5344988656079007
Weighted specificity: 0.9601067543307534
Weighted precision: 0.5344209812696527
Weighted F-score: 0.5330164053908905
Accuracy: 0.5344988656079007


In [13]:
## MODELO 8
Random.seed!(123);

parameters_ann = Dict("topology" => [15,15], "transferFunctions" => fill(sigmoid, 2), "learningRate" => 0.01, "repetitionsTraining" => 1, "validationRatio" => 0.0, "maxEpochsVal" => 20);

modelCrossValidation(:ANN, parameters_ann, inputs, targets, index)
#10 minitos


 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.8663285: validation loss : 2.8633735: test loss :2.8650148


Epoch 1000: train loss: 2.3900104: validation loss :2.48012: test loss :2.493968
 
Metrics: 
Weighted sensivity: 0.16946442845723422
Weighted specificity: 0.9259863861408886
Weighted precision: 0.16507468821350973
Weighted F-score: 0.15805803214501835
Accuracy: 0.16946442845723422
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.7744882: validation loss : 2.7721283: test loss :2.7738264


Epoch 736: train loss: 2.4032917: validation loss :2.4951267: test loss :2.5019777
 
Metrics: 
Weighted sensivity: 0.16109260493004662
Weighted specificity: 0.9233136515987147
Weighted precision: 0.1544363052521267
Weighted F-score: 0.14814196472757993
Accuracy: 0.16109260493004662
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.7629504: validation loss : 2.758717: test loss :2.762261


Epoch 907: train loss: 2.3966315: validation loss :2.4835737: test loss :2.4743516
 
Metrics: 
Weighted sensivity: 0.16375749500333112
Weighted specificity: 0.9246971466167503
Weighted precision: 0.15997118646429775
Weighted F-score: 0.15429984751387787
Accuracy: 0.16375749500333112
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.8146243: validation loss : 2.8170774: test loss :2.8141854


Epoch 817: train loss: 2.4065328: validation loss :2.5016708: test loss :2.4973617
 
Metrics: 
Weighted sensivity: 0.17135243171219186
Weighted specificity: 0.9240774979795626
Weighted precision: 0.1590480346082092
Weighted F-score: 0.15520342419543065
Accuracy: 0.17135243171219186
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.7290568: validation loss : 2.730864: test loss :2.7291808

...

Epoch 1000: train loss: 2.3864439: validation loss :2.5012805: test loss :2.504115
 
Metrics: 
Weighted sensivity: 0.16604427847425982
Weighted specificity: 0.9234059853581905
Weighted precision: 0.1579184864125012
Weighted F-score: 0.15185983506327969
Accuracy: 0.16604427847425982
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.698953: validation loss : 2.700393: test loss :2.6988268


Epoch 648: train loss: 2.3982344: validation loss :2.4921405: test loss :2.480093
 
Metrics: 
Weighted sensivity: 0.16971314209472982
Weighted specificity: 0.9238885116297841
Weighted precision: 0.1712089721721727
Weighted F-score: 0.15863123465687368
Accuracy: 0.16971314209472982
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 

0: train loss : 2.8060296: validation loss : 2.8084: test loss :2.8069003


Epoch 308: train loss: 2.4407408: validation loss :2.497751: test loss :2.501929
 
Metrics: 
Weighted sensivity: 0.150407046576805
Weighted specificity: 0.9200843312407152
Weighted precision: 0.14235644640683448
Weighted F-score: 0.13588637200509696
Accuracy: 0.150407046576805
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.7928271: validation loss : 2.7997038: test loss :2.7939517


Epoch 768: train loss: 2.4049199: validation loss :2.4979389: test loss :2.496832
 
Metrics: 
Weighted sensivity: 0.16054984652342186
Weighted specificity: 0.9233947846816722
Weighted precision: 0.15142396924629659
Weighted F-score: 0.1468911575954553
Accuracy: 0.16054984652342186


In [14]:
## MODELO 9
Random.seed!(123);

parameters_ann = Dict("topology" => [15,15], "transferFunctions" => fill(softmax, 2), "learningRate" => 0.01, "repetitionsTraining" => 1, "validationRatio" => 0.0, "maxEpochsVal" => 20);

modelCrossValidation(:ANN, parameters_ann, inputs, targets, index)
#10 minutos


 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.8663285: validation loss : 2.8633735: test loss :2.8650148


Epoch 1000: train loss: 2.3900104: validation loss :2.48012: test loss :2.493968
 
Metrics: 
Weighted sensivity: 0.16946442845723422
Weighted specificity: 0.9259863861408886
Weighted precision: 0.16507468821350973
Weighted F-score: 0.15805803214501835
Accuracy: 0.16946442845723422


 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.7744882: validation loss : 2.7721283: test loss :2.7738264


Epoch 736: train loss: 2.4032917: validation loss :2.4951267: test loss :2.5019777
 
Metrics: 
Weighted sensivity: 0.16109260493004662
Weighted specificity: 0.9233136515987147
Weighted precision: 0.1544363052521267
Weighted F-score: 0.14814196472757993
Accuracy: 0.16109260493004662
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.7629504: validation loss : 2.758717: test loss :2.762261


Epoch 907: train loss: 2.3966315: validation loss :2.4835737: test loss :2.4743516
 
Metrics: 
Weighted sensivity: 0.16375749500333112
Weighted specificity: 0.9246971466167503
Weighted precision: 0.15997118646429775
Weighted F-score: 0.15429984751387787
Accuracy: 0.16375749500333112
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------

...

Epoch 0: train loss : 2.698953: validation loss : 2.700393: test loss :2.6988268


Epoch 648: train loss: 2.3982344: validation loss :2.4921405: test loss :2.480093
 
Metrics: 
Weighted sensivity: 0.16971314209472982
Weighted specificity: 0.9238885116297841
Weighted precision: 0.1712089721721727
Weighted F-score: 0.15863123465687368
Accuracy: 0.16971314209472982
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.8060296: validation loss : 2.8084: test loss :2.8069003


Epoch 308: train loss: 2.4407408: validation loss :2.497751: test loss :2.501929
 
Metrics: 
Weighted sensivity: 0.150407046576805
Weighted specificity: 0.9200843312407152
Weighted precision: 0.14235644640683448
Weighted F-score: 0.13588637200509696
Accuracy: 0.150407046576805
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.7928271: validation loss : 2.7997038: test loss :2.7939517


Epoch 768: train loss: 2.4049199: validation loss :2.4979389: test loss :2.496832
 
Metrics: 
Weighted sensivity: 0.16054984652342186
Weighted specificity: 0.9233947846816722
Weighted precision: 0.15142396924629659
Weighted F-score: 0.1468911575954553
Accuracy: 0.16054984652342186


In [15]:
#MODELO 10
Random.seed!(123);

parameters_ann = Dict("topology" => [15,15], "transferFunctions" => fill(relu, 2), "learningRate" => 0.01, "repetitionsTraining" => 1, "validationRatio" => 0.0, "maxEpochsVal" => 20);

modelCrossValidation(:ANN, parameters_ann, inputs, targets, index)
#10 minutos

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.8663285: validation loss : 2.8633735: test loss :2.8650148


Epoch 1000: train loss: 2.3900104: validation loss :2.48012: test loss :2.493968
 
Metrics: 
Weighted sensivity: 0.16946442845723422
Weighted specificity: 0.9259863861408886
Weighted precision: 0.16507468821350973
Weighted F-score: 0.15805803214501835
Accuracy: 0.16946442845723422
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.7744882: validation loss : 2.7721283: test loss :2.7738264


Epoch 736: train loss: 2.4032917: validation loss :2.4951267: test loss :2.5019777
 
Metrics: 
Weighted sensivity: 0.16109260493004662
Weighted specificity: 0.9233136515987147
Weighted precision: 0.1544363052521267
Weighted F-score: 0.14814196472757993
Accuracy: 0.16109260493004662
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.7629504: validation loss : 2.758717: test loss :2.762261


Epoch 907: train loss: 2.3966315: validation loss :2.4835737: test loss :2.4743516
 
Metrics: 
Weighted sensivity: 0.16375749500333112
Weighted specificity: 0.9246971466167503
Weighted precision: 0.15997118646429775
Weighted F-score: 0.15429984751387787
Accuracy: 0.16375749500333112
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.8146243: validation loss : 2.8170774: test loss :2.8141854


Epoch 817: train loss: 2.4065328: validation loss :2.5016708: test loss :2.4973617
 
Metrics: 
Weighted sensivity: 0.17135243171219186
Weighted specificity: 0.9240774979795626
Weighted precision: 0.1590480346082092
Weighted F-score: 0.15520342419543065
Accuracy: 0.17135243171219186
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------

...

Epoch 1000: train loss: 2.3864439: validation loss :2.5012805: test loss :2.504115
 
Metrics: 
Weighted sensivity: 0.16604427847425982
Weighted specificity: 0.9234059853581905
Weighted precision: 0.1579184864125012
Weighted F-score: 0.15185983506327969
Accuracy: 0.16604427847425982
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.698953: validation loss : 2.700393: test loss :2.6988268


Epoch 648: train loss: 2.3982344: validation loss :2.4921405: test loss :2.480093
 
Metrics: 
Weighted sensivity: 0.16971314209472982
Weighted specificity: 0.9238885116297841
Weighted precision: 0.1712089721721727
Weighted F-score: 0.15863123465687368
Accuracy: 0.16971314209472982
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.8060296: validation loss : 2.8084: test loss :2.8069003


Epoch 308: train loss: 2.4407408: validation loss :2.497751: test loss :2.501929
 
Metrics: 
Weighted sensivity: 0.150407046576805
Weighted specificity: 0.9200843312407152
Weighted precision: 0.14235644640683448
Weighted F-score: 0.13588637200509696
Accuracy: 0.150407046576805
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.7928271: validation loss : 2.7997038: test loss :2.7939517


Epoch 768: train loss: 2.4049199: validation loss :2.4979389: test loss :2.496832
 
Metrics: 
Weighted sensivity: 0.16054984652342186
Weighted specificity: 0.9233947846816722
Weighted precision: 0.15142396924629659
Weighted F-score: 0.1468911575954553
Accuracy: 0.16054984652342186


In [16]:
#MODELO 11
Random.seed!(123);


parameters_ann = Dict("topology" => [15,15], "transferFunctions" => fill(tanh, 2), "learningRate" => 0.01, "repetitionsTraining" => 1, "validationRatio" => 0.0, "maxEpochsVal" => 20);

modelCrossValidation(:ANN, parameters_ann, inputs, targets, index)
#10 minutos


------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.8663285: validation loss : 2.8633735: test loss :2.8650148


Epoch 1000: train loss: 2.3900104: validation loss :2.48012: test loss :2.493968
 
Metrics: 
Weighted sensivity: 0.16946442845723422
Weighted specificity: 0.9259863861408886
Weighted precision: 0.16507468821350973
Weighted F-score: 0.15805803214501835
Accuracy: 0.16946442845723422
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.7744882: validation loss : 2.7721283: test loss :2.7738264


Epoch 736: train loss: 2.4032917: validation loss :2.4951267: test loss :2.5019777
 
Metrics: 
Weighted sensivity: 0.16109260493004662
Weighted specificity: 0.9233136515987147
Weighted precision: 0.1544363052521267
Weighted F-score: 0.14814196472757993
Accuracy: 0.16109260493004662
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.7629504: validation loss : 2.758717: test loss :2.762261


Epoch 907: train loss: 2.3966315: validation loss :2.4835737: test loss :2.4743516
 
Metrics: 
Weighted sensivity: 0.16375749500333112
Weighted specificity: 0.9246971466167503
Weighted precision: 0.15997118646429775
Weighted F-score: 0.15429984751387787
Accuracy: 0.16375749500333112
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 

2.8146243: validation loss : 2.8170774: test loss :2.8141854
Epoch 817: train loss: 2.4065328: validation loss :2.5016708: test loss :2.4973617
 
Metrics: 
Weighted sensivity: 0.17135243171219186
Weighted specificity: 0.9240774979795626
Weighted precision: 0.1590480346082092
Weighted F-score: 0.15520342419543065
Accuracy: 0.17135243171219186
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


...

Epoch 1000: train loss: 2.3864439: validation loss :2.5012805: test loss :2.504115
 
Metrics: 
Weighted sensivity: 0.16604427847425982
Weighted specificity: 0.9234059853581905
Weighted precision: 0.1579184864125012
Weighted F-score: 0.15185983506327969
Accuracy: 0.16604427847425982
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.698953: validation loss : 

2.700393: test loss :2.6988268


Epoch 648: train loss: 2.3982344: validation loss :2.4921405: test loss :2.480093
 
Metrics: 
Weighted sensivity: 0.16971314209472982
Weighted specificity: 0.9238885116297841
Weighted precision: 0.1712089721721727
Weighted F-score: 0.15863123465687368
Accuracy: 0.16971314209472982
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.8060296: validation loss : 2.8084: test loss :2.8069003


Epoch 308: train loss: 2.4407408: validation loss :2.497751: test loss :2.501929
 
Metrics: 
Weighted sensivity: 0.150407046576805
Weighted specificity: 0.9200843312407152
Weighted precision: 0.14235644640683448
Weighted F-score: 0.13588637200509696
Accuracy: 0.150407046576805
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.7928271: validation loss : 2.7997038: test loss :2.7939517


Epoch 768: train loss: 2.4049199: validation loss :2.4979389: test loss :2.496832
 
Metrics: 
Weighted sensivity: 0.16054984652342186
Weighted specificity: 0.9233947846816722
Weighted precision: 0.15142396924629659
Weighted F-score: 0.1468911575954553
Accuracy: 0.16054984652342186


In [17]:
#MODELO 12
Random.seed!(123);

parameters_ann = Dict("topology" => [120,120], "transferFunctions" => fill(relu, 2), "learningRate" => 0.01, "repetitionsTraining" => 1, "validationRatio" => 0.0, "maxEpochsVal" => 20);

modelCrossValidation(:ANN, parameters_ann, inputs, targets, index)
# 52 minutos

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 3.0026958: validation loss : 

3.0044923: test loss :3.0030756
Epoch 1000: train loss: 0.8703592: validation loss :1.8923202: test loss :1.8791553
 
Metrics: 
Weighted sensivity: 0.5907274180655475
Weighted specificity: 0.9655448933216755
Weighted precision: 0.5903013983344398
Weighted F-score: 0.5891487436170533
Accuracy: 0.5907274180655475
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 3.074513: validation loss : 3.0833604: test loss :3.0769217


Epoch 1000: train loss: 0.8799298: validation loss :1.866853: test loss :1.9212604
 
Metrics: 
Weighted sensivity: 0.5849433710859427
Weighted specificity: 0.9648069249076359
Weighted precision: 0.5834634088345533
Weighted F-score: 0.5825115121608203
Accuracy: 0.5849433710859427
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------

...

Epoch 0: train loss : 3.053083: validation loss : 3.0450542: test loss :3.0515814


Epoch 1000: train loss: 0.82799864: validation loss :1.9261812: test loss :1.843443
 
Metrics: 
Weighted sensivity: 0.5992259442146003
Weighted specificity: 0.9664282540074866
Weighted precision: 0.5989763011773442
Weighted F-score: 0.5984078722842625
Accuracy: 0.5992259442146003
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------


------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.7216406: validation loss : 2.7307165: test loss :2.7235134


Epoch 961: train loss: 0.85280293: validation loss :1.8575989: test loss :1.8928643
 
Metrics: 
Weighted sensivity: 0.5798745495796076
Weighted specificity: 0.9647685986129142
Weighted precision: 0.5796925897067127
Weighted F-score: 0.5793040894089334
Accuracy: 0.5798745495796076


In [18]:
#MODELO 13
Random.seed!(123);

parameters_ann = Dict("topology" => [240,240], "transferFunctions" => fill(relu, 2), "learningRate" => 0.01, "repetitionsTraining" => 1, "validationRatio" => 0.0, "maxEpochsVal" => 20);

modelCrossValidation(:ANN, parameters_ann, inputs, targets, index)
#100 mins

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.7638445: validation loss : 2.7578304: test loss :2.7631292


Epoch 1000: train loss: 0.22410567: validation loss :1.61502: test loss :1.6658386
 
Metrics: 
Weighted sensivity: 0.7871036504130029
Weighted specificity: 0.9822021509674748
Weighted precision: 0.7877341373926413
Weighted F-score: 0.7869717968728189
Accuracy: 0.7871036504130029
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.925189: validation loss : 2.9249876: test loss :2.9244623


Epoch 982: train loss: 0.21367067: validation loss :1.6202699: test loss :1.6464958
 
Metrics: 
Weighted sensivity: 0.7792138574283811
Weighted specificity: 0.9817370674825492
Weighted precision: 0.7793324890489994
Weighted F-score: 0.7790530991711251
Accuracy: 0.7792138574283811
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------

...


Epoch 0: train loss : 3.0734851: validation loss : 3.0619574: test loss :3.0717957


Epoch 971: train loss: 0.2272297: validation loss :1.637206: test loss :1.5945532
 
Metrics: 
Weighted sensivity: 0.7805952222073935
Weighted specificity: 0.9815370063099738


Weighted precision: 0.780722834927478
Weighted F-score: 0.7804463058997764
Accuracy: 0.7805952222073935
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------


------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.9737651: validation loss : 2.9833624: test loss :2.976871


Epoch 1000: train loss: 0.25431916: validation loss :1.6826116: test loss :1.6838453
 
Metrics: 
Weighted sensivity: 0.7700520485786734
Weighted specificity: 0.9807189053256544
Weighted precision: 0.7702174279098757
Weighted F-score: 0.7699224980440188
Accuracy: 0.7700520485786734


In [19]:
#MODELO 14
Random.seed!(123);

parameters_ann = Dict("topology" => [240,240], "transferFunctions" => [sigmoid, relu], "learningRate" => 0.01, "repetitionsTraining" => 1, "validationRatio" => 0.0, "maxEpochsVal" => 20);

modelCrossValidation(:ANN, parameters_ann, inputs, targets, index)
#Tiempo aprox 40 min por fold y f1 aprox 0.77
#80 minutos

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------


------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.7638445: validation loss : 2.7578304: test loss :2.7631292


Epoch 1000: train loss: 0.22410567: validation loss :1.61502: test loss :1.6658386
 
Metrics: 
Weighted sensivity: 0.7871036504130029
Weighted specificity: 0.9822021509674748
Weighted precision: 0.7877341373926413
Weighted F-score: 0.7869717968728189
Accuracy: 0.7871036504130029
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.925189: validation loss : 2.9249876: test loss :2.9244623


Epoch 982: train loss: 0.21367067: validation loss :1.6202699: test loss :1.6464958
 
Metrics: 
Weighted sensivity: 0.7792138574283811
Weighted specificity: 0.9817370674825492
Weighted precision: 0.7793324890489994
Weighted F-score: 0.7790530991711251
Accuracy: 0.7792138574283811
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------

...

Epoch 1000: train loss: 0.22779486: validation loss :1.5706927: test loss :1.5606389
 
Metrics: 
Weighted sensivity: 0.7911941294196131
Weighted specificity: 0.982342427431282
Weighted precision: 0.7914810087126577
Weighted F-score: 0.7910981435665486
Accuracy: 0.7911941294196131
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 3.0734851: validation loss : 3.0619574: test loss :3.0717957


Epoch 971: train loss: 0.2272297: validation loss :1.637206: test loss :1.5945532
 
Metrics: 
Weighted sensivity: 0.7805952222073935
Weighted specificity: 0.9815370063099738
Weighted precision: 0.780722834927478
Weighted F-score: 0.7804463058997764
Accuracy: 0.7805952222073935
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.9737651: validation loss : 2.9833624: test loss :2.976871


Epoch 1000: train loss: 0.25431916: validation loss :1.6826116: test loss :1.6838453
 
Metrics: 
Weighted sensivity: 0.7700520485786734
Weighted specificity: 0.9807189053256544
Weighted precision: 0.7702174279098757
Weighted F-score: 0.7699224980440188
Accuracy: 0.7700520485786734


In [20]:
#MODELO 15
Random.seed!(123);

parameters_ann = Dict("topology" => [480,240], "transferFunctions" => [sigmoid, relu], "learningRate" => 0.01, "repetitionsTraining" => 1, "validationRatio" => 0.0, "maxEpochsVal" => 20);

modelCrossValidation(:ANN, parameters_ann, inputs, targets, index)
#60 mins

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.7741506: validation loss : 2.7664988: test loss :2.7727861


Epoch 500: train loss: 0.613347: validation loss :1.5743495: test loss :1.5819545
 
Metrics: 
Weighted sensivity: 0.6755928590460965
Weighted specificity: 0.9730210917895278
Weighted precision: 0.6755664533848544
Weighted F-score: 0.6740940983547469
Accuracy: 0.6755928590460965
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------


------------------------------------------------------------------------------------------------------------------
Epoch 0: train loss : 2.9436815: validation loss : 2.9439807: test loss :2.943855


Epoch 498: train loss: 0.7572879: validation loss :1.6823831: test loss :1.702472
 
Metrics: 
Weighted sensivity: 0.638107928047968
Weighted specificity: 0.9690011053141935
Weighted precision: 0.6381228752450855
Weighted F-score: 0.6349005292481651
Accuracy: 0.638107928047968
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------


...

Epoch 0: train loss : 3.142763: validation loss : 3.1305902: test loss :3.140469


Epoch 501: train loss: 0.5579525: validation loss :1.5363843: test loss :1.5315648
 
Metrics: 
Weighted sensivity: 0.6799679701054318
Weighted specificity: 0.9729552900497355
Weighted precision: 0.6806808562772151
Weighted F-score: 0.6795948535875237
Accuracy: 0.6799679701054318


 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


Epoch 0: train loss : 2.9614203: validation loss : 2.9703329: test loss :2.9625962


Epoch 450: train loss: 0.66711843: validation loss :1.5730516: test loss :1.5583968
 
Metrics: 
Weighted sensivity: 0.6388629387428266
Weighted specificity: 0.9700914451969515
Weighted precision: 0.6379622729140921
Weighted F-score: 0.6374319555385995
Accuracy: 0.6388629387428266


### SVM (8 variando los kernel y c)

Podemos variar en los modelos el valor de c y el valor del kernel. Para el valor de c, cuanto mayor sea, menos error en el modelo (mas posibilidades de overfitting) y menos margen. Con un c pequeño pocas posibilidades de conseguir overfitting pero mas error. A mayores tenemos los diferentes kernels que podemos usar: ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’ (lineal, polinomial, rbf y sigmoidal). Para rbf podemos definir la constante gamma, para los modelos polinomiales podemos especificar d(grado).

In [46]:
include("functions_finalproject.jl")

trainClassEnsemble (generic function with 1 method)

In [21]:
## MODELO 1
# Using random seed so we can repeat the results

parameters_svm = Dict("kernel" => "rbf", "kernelDegree" => 3, "kernelGamma" => 3, "C" => 1); #kernel = ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’
#el degree no es necesario (estamos en el caso de rbf, por como esta definida la funcion es necesario implementarlo pero no se usa)

modelos = modelCrossValidation(:SVM, parameters_svm, inputs, targets, index)

# 120 mins

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8729016786570744
Weighted specificity: 0.9848164421482948
Weighted precision: 0.9419910225665622
Weighted F-score: 0.8916710930525532
Accuracy: 0.8729016786570744
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8664890073284477
Weighted specificity: 0.9840479855116477
Weighted precision: 0.9406864641542355
Weighted F-score: 0.8867296928547271
Accuracy: 0.8664890073284477
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8723517654896735
Weighted specificity: 0.984748473173811
Weighted precision: 0.9418725208398115
Weighted F-score: 0.8912469961369088
Accuracy: 0.8723517654896735
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.870219853431046
Weighted specificity: 0.9844937503875698
Weighted precision: 0.9414344239990242
Weighted F-score: 0.8895904775069067
Accuracy: 0.870219853431046
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


...


 
Metrics: 
Weighted sensivity: 0.8749166555540738
Weighted specificity: 0.9850624458043378
Weighted precision: 0.942424237309125
Weighted F-score: 0.8932042258233944
Accuracy: 0.8749166555540738
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8698319551880501
Weighted specificity: 0.9844609413722933
Weighted precision: 0.9412409700353652
Weighted F-score: 0.8892608844634311
Accuracy: 0.8698319551880501
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8699132755170114
Weighted specificity: 0.9844556565218237
Weighted precision: 0.9413693636133009
Weighted F-score: 0.8893691648432709
Accuracy: 0.8699132755170114
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8724142533030829
Weighted specificity: 0.9847499481013696
Weighted precision: 0.9418743750811311
Weighted F-score: 0.8913127624037107
Accuracy: 0.8724142533030829
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8704123848925664
Weighted specificity: 0.9845106690443827
Weighted precision: 0.9414623985963033
Weighted F-score: 0.8897184777509336
Accuracy: 0.8704123848925664


In [22]:
## MODELO 2

parameters_svm = Dict("kernel" => "linear", "kernelDegree" => 3, "kernelGamma" => 2, "C" => 1); #kernel = ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’

modelCrossValidation(:SVM, parameters_svm, inputs, targets, index)

#F score 0.1186 con un tiempo de 26 minutos (ordenador de la facultad).
#240 mins

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.1442845723421263
Weighted specificity: 0.9102216099807331
Weighted precision: 0.1249435056249624
Weighted F-score: 0.1120755549013811
Accuracy: 0.1442845723421263
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.14177215189873418
Weighted specificity: 0.9092524752928748
Weighted precision: 0.1419609328901455
Weighted F-score: 0.10982488634047972
Accuracy: 0.14177215189873418
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.14763491005996002
Weighted specificity: 0.9102128115436275
Weighted precision: 0.14494801588368555
Weighted F-score: 0.11361445340258434
Accuracy: 0.14763491005996002
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.14510326449033978
Weighted specificity: 0.9099824302414754
Weighted precision: 0.1442444742346456
Weighted F-score: 0.11304019100320413
Accuracy: 0.14510326449033978
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------

...


------------------------------------------------------------------------------------------------------------------
 
Metrics: 
Weighted sensivity: 0.14588611814908656
Weighted specificity: 0.909855613848602
Weighted precision: 0.13707239370748248
Weighted F-score: 0.11230341693540379
Accuracy: 0.14588611814908656
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.14830621499066418
Weighted specificity: 0.910620146513524
Weighted precision: 0.1519406901627787
Weighted F-score: 0.11508267154798094
Accuracy: 0.14830621499066418
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.14876584389593062
Weighted specificity: 0.9100546821431275
Weighted precision: 0.14167620061282585
Weighted F-score: 0.11454894434938864
Accuracy: 0.14876584389593062
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.1474709729080475
Weighted specificity: 0.9102773103556929
Weighted precision: 0.13171880520875373
Weighted F-score: 0.11472489917720095
Accuracy: 0.1474709729080475
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.1453356466034966
Weighted specificity: 0.9098056885919226
Weighted precision: 0.15995918859660166
Weighted F-score: 0.11054620318188257
Accuracy: 0.1453356466034966


In [23]:
## MODELO 3

parameters_svm = Dict("kernel" => "poly", "kernelDegree" => 3, "kernelGamma" => 2, "C" => 1); #kernel = ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’

modelCrossValidation(:SVM, parameters_svm, inputs, targets, index)

#F score: 0.8751 con un tiempo de 6 mins y medio (ordenador de la facultad)
#50 mins

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.882494004796163
Weighted specificity: 0.9909122575007262
Weighted precision: 0.8834278797602308
Weighted F-score: 0.8827336958491014
Accuracy: 0.882494004796163
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8766155896069288
Weighted specificity: 0.9903980154666786
Weighted precision: 0.8771918862927285
Weighted F-score: 0.8767560484000453
Accuracy: 0.8766155896069288
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.880346435709527
Weighted specificity: 0.9906698949460095
Weighted precision: 0.8812734515463984
Weighted F-score: 0.8805205935567245
Accuracy: 0.880346435709527
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8822118587608261
Weighted specificity: 0.9908410381048113
Weighted precision: 0.8831975149643414
Weighted F-score: 0.8824026285442498
Accuracy: 0.8822118587608261
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


...

 
Metrics: 
Weighted sensivity: 0.8854513935191359
Weighted specificity: 0.9912325827680257
Weighted precision: 0.8863127107524214
Weighted F-score: 0.8856047519173621
Accuracy: 0.8854513935191359
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8782341957855428
Weighted specificity: 0.9904783079341903
Weighted precision: 0.879004667079406
Weighted F-score: 0.8783930579119921
Accuracy: 0.8782341957855428
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8811207471647765
Weighted specificity: 0.9905230278909479
Weighted precision: 0.8817090949186345
Weighted F-score: 0.8811920058962098
Accuracy: 0.8811207471647765
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8812224743093554
Weighted specificity: 0.9908584083579074
Weighted precision: 0.8824194741129695
Weighted F-score: 0.881452561389283
Accuracy: 0.8812224743093554
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8805551848391833
Weighted specificity: 0.9905728624615223
Weighted precision: 0.8810065460237475
Weighted F-score: 0.8806501660594649
Accuracy: 0.8805551848391833


In [24]:
## MODELO 4

parameters_svm = Dict("kernel" => "sigmoid", "kernelDegree" => 3, "kernelGamma" => 2, "C" => 1); #kernel = ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’

modelCrossValidation(:SVM, parameters_svm, inputs, targets, index)

# 50 mins

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.05928590460964562
Weighted specificity: 0.9372000630645403
Weighted precision: 0.07084776404199862
Weighted F-score: 0.05542280825044737
Accuracy: 0.05928590460964562
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.07634910059960026
Weighted specificity: 0.9164322308040319
Weighted precision: 0.0663186977961422
Weighted F-score: 0.0534425627981639
Accuracy: 0.07634910059960026
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.059427048634243834
Weighted specificity: 0.9341787211049489
Weighted precision: 0.06319272579358967
Weighted F-score: 0.050066170906302915
Accuracy: 0.059427048634243834
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.07861425716189208
Weighted specificity: 0.9172731568248979
Weighted precision: 0.07374381873096658
Weighted F-score: 0.058658952472247776
Accuracy: 0.07861425716189208
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


...


 
Metrics: 
Weighted sensivity: 0.0661421522869716
Weighted specificity: 0.926445438642701
Weighted precision: 0.062192697335912935
Weighted F-score: 0.05520289530201882
Accuracy: 0.0661421522869716
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.06815150706855161
Weighted specificity: 0.9214589850182562
Weighted precision: 0.06042698348961943
Weighted F-score: 0.05517711042627082
Accuracy: 0.06815150706855161
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.06270847231487658
Weighted specificity: 0.9295324112026946
Weighted precision: 0.07125903745757214
Weighted F-score: 0.051816957537170825
Accuracy: 0.06270847231487658
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.07607099959962632
Weighted specificity: 0.9186652587332357
Weighted precision: 0.07437003186971529
Weighted F-score: 0.06300726305424433
Accuracy: 0.07607099959962632
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.07553716802348859
Weighted specificity: 0.9183097761281119
Weighted precision: 0.06205765774760082
Weighted F-score: 0.053033397392110784
Accuracy: 0.07553716802348859


Viendo los cuatro modelos en principio los dos mejores modelos son los modelos lineales y los modelos polinomicos. 

In [25]:
## MODELO 5

parameters_svm = Dict("kernel" => "poly", "kernelDegree" => 2, "kernelGamma" => 2, "C" => 1); #kernel = ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’

modelCrossValidation(:SVM, parameters_svm, inputs, targets, index)

# 290 mins

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8788968824940048
Weighted specificity: 0.9907061536602586
Weighted precision: 0.8802283105864558
Weighted F-score: 0.879172113921474
Accuracy: 0.8788968824940048
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8735509660226516
Weighted specificity: 0.9900839872184669
Weighted precision: 0.8742828482064109
Weighted F-score: 0.8736859062255431
Accuracy: 0.8735509660226516
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8752831445702864
Weighted specificity: 0.9905194924779043
Weighted precision: 0.8769355314651195
Weighted F-score: 0.8756028356564006
Accuracy: 0.8752831445702864
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8760826115922719
Weighted specificity: 0.9905344822702027
Weighted precision: 0.8773817712211317
Weighted F-score: 0.8763352871397184
Accuracy: 0.8760826115922719
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------

...

 
Metrics: 
Weighted sensivity: 0.8822509667955727
Weighted specificity: 0.9908507519046936
Weighted precision: 0.8832342430611615
Weighted F-score: 0.8824314161577654
Accuracy: 0.8822509667955727
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.875433448919712
Weighted specificity: 0.9904916639938526
Weighted precision: 0.8766010109916449
Weighted F-score: 0.8757023203420627
Accuracy: 0.875433448919712
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8747164776517679
Weighted specificity: 0.990463671866838
Weighted precision: 0.8757479029284196
Weighted F-score: 0.874897761017342
Accuracy: 0.8747164776517679
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.878019484852529
Weighted specificity: 0.9909207035724105
Weighted precision: 0.8799780854179241
Weighted F-score: 0.8784380881406517
Accuracy: 0.878019484852529
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8750834111837715
Weighted specificity: 0.9903877010600355
Weighted precision: 0.8760496307540987
Weighted F-score: 0.8753156460378168
Accuracy: 0.8750834111837715


In [26]:
## MODELO 6

parameters_svm = Dict("kernel" => "poly", "kernelDegree" => 5, "kernelGamma" => 2, "C" => 1); #kernel = ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’

modelCrossValidation(:SVM, parameters_svm, inputs, targets, index)

# 70 mins

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8820943245403677
Weighted specificity: 0.9904004414593562
Weighted precision: 0.8831823368851116
Weighted F-score: 0.8823250577336336
Accuracy: 0.8820943245403677


 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8782145236508994
Weighted specificity: 0.9898321355597499
Weighted precision: 0.8794604447885341
Weighted F-score: 0.8784560619311305
Accuracy: 0.8782145236508994
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.880346435709527
Weighted specificity: 0.9901382508578108
Weighted precision: 0.8814126427089286
Weighted F-score: 0.8805341617998661
Accuracy: 0.880346435709527
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------

...

 
Metrics: 
Weighted sensivity: 0.8850513401786905
Weighted specificity: 0.9905293819661803
Weighted precision: 0.8856318307753684
Weighted F-score: 0.885059373847867
Accuracy: 0.8850513401786905
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.87810082688717
Weighted specificity: 0.989999537612609
Weighted precision: 0.8794252230873001
Weighted F-score: 0.8782976048008487
Accuracy: 0.87810082688717
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8799199466310874
Weighted specificity: 0.9900413753206112
Weighted precision: 0.8806810889051853
Weighted F-score: 0.8800686563545854
Accuracy: 0.8799199466310874
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8824235953556653
Weighted specificity: 0.9905603136633951
Weighted precision: 0.8834915411358897
Weighted F-score: 0.8826350057521839
Accuracy: 0.8824235953556653
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8821566795675965
Weighted specificity: 0.9902059708963173
Weighted precision: 0.8827963291741607
Weighted F-score: 0.8822156090207137
Accuracy: 0.8821566795675965


En los polinomicos no mejora al aumentar el grado del polinomio, en ppio lo optimo sería tener un modelo polinomial de grado 2 o 3.

In [27]:
## MODELO 7

parameters_svm = Dict("kernel" => "rbf", "kernelDegree" => 3, "kernelGamma" => 20, "C" => 1000); #kernel = ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’

modelCrossValidation(:SVM, parameters_svm, inputs, targets, index)
#120 mins

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8726352251532108
Weighted specificity: 0.9847846107901151
Weighted precision: 0.9419355807329094
Weighted F-score: 0.8914636740563928
Accuracy: 0.8726352251532108
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8655562958027981
Weighted specificity: 0.9839365442926673
Weighted precision: 0.9405030900210173
Weighted F-score: 0.8860113779170519
Accuracy: 0.8655562958027981
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8716855429713525
Weighted specificity: 0.9846688723031105
Weighted precision: 0.9417347618594406
Weighted F-score: 0.890732366011188
Accuracy: 0.8716855429713525
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8696868754163891
Weighted specificity: 0.9844300696910094
Weighted precision: 0.9413261310896727
Weighted F-score: 0.8891809859588166
Accuracy: 0.8696868754163891
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------

...


 
Metrics: 
Weighted sensivity: 0.8743832511001467
Weighted specificity: 0.9849987462128851
Weighted precision: 0.9423114815614911
Weighted F-score: 0.8927833795075308
Accuracy: 0.8743832511001467
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8691651106961856
Weighted specificity: 0.9843732589664002
Weighted precision: 0.9412308189539296
Weighted F-score: 0.888791113272305
Accuracy: 0.8691651106961856
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8687124749833223
Weighted specificity: 0.9843121702743328
Weighted precision: 0.9411266703961086
Weighted F-score: 0.888428294420132
Accuracy: 0.8687124749833223
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.872147337515014
Weighted specificity: 0.9847180442271045
Weighted precision: 0.9418190386871508
Weighted F-score: 0.8911040646996514
Accuracy: 0.872147337515014
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8694781796343254
Weighted specificity: 0.9843990054844555
Weighted precision: 0.9412725217702251
Weighted F-score: 0.888989530458464
Accuracy: 0.8694781796343254


In [28]:
## MODELO 8

parameters_svm = Dict("kernel" => "sigmoid", "kernelDegree" => 3, "kernelGamma" => 2, "C" => 1000); #kernel = ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’

modelCrossValidation(:SVM, parameters_svm, inputs, targets, index)

# 50 mins

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------


------------------------------------------------------------------------------------------------------------------
 
Metrics: 
Weighted sensivity: 0.058886224353850256
Weighted specificity: 0.9368644585552639
Weighted precision: 0.07028348961985292
Weighted F-score: 0.05459659766758109
Accuracy: 0.058886224353850256


 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.0787475016655563
Weighted specificity: 0.916708078468278
Weighted precision: 0.06687893164083845
Weighted F-score: 0.05517130126390952
Accuracy: 0.0787475016655563
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.0608927381745503
Weighted specificity: 0.9343887330605629
Weighted precision: 0.06508503143380726
Weighted F-score: 0.0518250662848362
Accuracy: 0.0608927381745503
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


...


 
Metrics: 
Weighted sensivity: 0.06654220562741699
Weighted specificity: 0.9265908140862433
Weighted precision: 0.06442201707362151
Weighted F-score: 0.055796342660772456
Accuracy: 0.06654220562741699
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.0704187783408909
Weighted specificity: 0.9219144055543204
Weighted precision: 0.06520015178652637
Weighted F-score: 0.058199978435600604
Accuracy: 0.0704187783408909
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.0628418945963976
Weighted specificity: 0.9292851009424339
Weighted precision: 0.06972774724228283
Weighted F-score: 0.050808048496266224
Accuracy: 0.0628418945963976
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.07687174696383291
Weighted specificity: 0.9190038595755016
Weighted precision: 0.07592698087573926
Weighted F-score: 0.0639125315741373
Accuracy: 0.07687174696383291
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.07540371012945415
Weighted specificity: 0.918441003694903
Weighted precision: 0.06158250764356378
Weighted F-score: 0.05316745128153486
Accuracy: 0.07540371012945415


In [ ]:
#Como abrir los modelos (los outputs pq el modelo da error)

folder_path = "modelsvm rbf, 1, 3, 3"

# File path to load the model from inside the folder
model_file_path = joinpath(folder_path, "targets_svmmodel1.jld2")

targets_model = nothing

# Load the model from the JLD file
loaded_model = JLD2.jldopen(model_file_path) do file
     global targets_model
     targets_model = read(file, "targets_test")
end

model2_file_path = joinpath(folder_path, "outputs_svmmodel1.jld2")

outputs_model = nothing

# Load the model from the JLD file
loaded_model = JLD2.jldopen(model2_file_path) do file
     global outputs_model
     outputs_model = read(file, "outputs_model")
end

confusionMatrix(outputs_model, targets_model)


Claramente los mejores modelos a la hora de los resultados son los polinomiales y los lineales (polinomiales de grado 2 o 3) ya que aun aumentando drasticamente el valor de c no conseguimos un f-score y una sensitividad similar a los otros dos modelos. Claramente para este problema de clasificacion SVM trabaja mucho mejor que una ANN (llegando a conseguir una fscore de 40% mientras que en el mejor ANN conseguimos un 10% a lo sumo).

### kNN (6 modelos variando los k)

In [6]:
##MODEL 1

parameters_knn = Dict("n_neighbors" => 3);

modelCrossValidation(:kNN, parameters_knn, inputs, targets, index) 

## Fold 8: Tiempo inmediato, F-score 0.755

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.7438390835220461
Weighted specificity: 0.979947728257397
Weighted precision: 0.7460199653422659
Weighted F-score: 0.7439199612433405
Accuracy: 0.7438390835220461
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.7429713524317122
Weighted specificity: 0.9798515567227631
Weighted precision: 0.7455306498295939
Weighted F-score: 0.7434617185736547
Accuracy: 0.7429713524317122
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.7431351639562783
Weighted specificity: 0.979679053289089
Weighted precision: 0.7454685352688256
Weighted F-score: 0.7433926203980616
Accuracy: 0.7431351639562783
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.7512331689108119
Weighted specificity: 0.9803443858039285
Weighted precision: 0.7531046035250194
Weighted F-score: 0.7513528508225813
Accuracy: 0.7512331689108119
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


...

 
Metrics: 
Weighted sensivity: 0.7362666666666666
Weighted specificity: 0.9789347696179818
Weighted precision: 0.737397061728698
Weighted F-score: 0.7363641583134194
Accuracy: 0.7362666666666666
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.7446326176823577
Weighted specificity: 0.9797389433880419
Weighted precision: 0.7461595021453845
Weighted F-score: 0.7447626693732619
Accuracy: 0.7446326176823577
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.7478991596638656
Weighted specificity: 0.9799304798076698
Weighted precision: 0.7500812761612418
Weighted F-score: 0.7482341592850688
Accuracy: 0.7478991596638656
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.751567711807872
Weighted specificity: 0.9804600731424418
Weighted precision: 0.7547549655784757
Weighted F-score: 0.7519662786000428
Accuracy: 0.751567711807872
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.7485988791032826
Weighted specificity: 0.9801940770308988
Weighted precision: 0.7512417288925927
Weighted F-score: 0.7489454652333296
Accuracy: 0.7485988791032826


In [7]:
##MODEL 2

parameters_knn = Dict("n_neighbors" => 5);

modelCrossValidation(:kNN, parameters_knn, inputs, targets, index)

#Tiempo inmediato, F-score = 0.633 Fold 8

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.6267483681896896
Weighted specificity: 0.9702746842655424
Weighted precision: 0.6291375045165292
Weighted F-score: 0.626719224113985
Accuracy: 0.6267483681896896
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.6162558294470353
Weighted specificity: 0.9694264425128665
Weighted precision: 0.6196805421953484
Weighted F-score: 0.6167248589395398
Accuracy: 0.6162558294470353
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.6143695014662757
Weighted specificity: 0.9692482931033982
Weighted precision: 0.6173420319850454
Weighted F-score: 0.6142692178025874
Accuracy: 0.6143695014662757
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.626449806692441
Weighted specificity: 0.970117512816142
Weighted precision: 0.6291962429289631
Weighted F-score: 0.6265519436858041
Accuracy: 0.626449806692441
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


...

 
Metrics: 
Weighted sensivity: 0.6116
Weighted specificity: 0.9688875235906876
Weighted precision: 0.6135959374950695
Weighted F-score: 0.6114537147758003
Accuracy: 0.6116
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.6106147486331511
Weighted specificity: 0.9689818564627998
Weighted precision: 0.6135230533664386
Weighted F-score: 0.6107570125012655
Accuracy: 0.6106147486331511
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.6202480992396959
Weighted specificity: 0.969694565194507
Weighted precision: 0.6241149111909223
Weighted F-score: 0.620620535104499
Accuracy: 0.6202480992396959
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.6313542361574382
Weighted specificity: 0.9707807840398498
Weighted precision: 0.6356658495121033
Weighted F-score: 0.6317194098243976
Accuracy: 0.6313542361574382
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.6203629570322925
Weighted specificity: 0.9696241708072024
Weighted precision: 0.62364190561784
Weighted F-score: 0.620695592768941
Accuracy: 0.6203629570322925


In [8]:
##MODEL 3

parameters_knn = Dict("n_neighbors" => 10);

modelCrossValidation(:kNN, parameters_knn, inputs, targets, index)

#Tiempo inmediato, F-score 0.457, Fold 7

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.45624084188091113
Weighted specificity: 0.9566470615884675
Weighted precision: 0.45930180580845825
Weighted F-score: 0.4560450455446066
Accuracy: 0.45624084188091113
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.44596935376415725
Weighted specificity: 0.955416562171529
Weighted precision: 0.4495826192171958
Weighted F-score: 0.44614779812059485
Accuracy: 0.44596935376415725
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.4413489736070381
Weighted specificity: 0.9553617982837386
Weighted precision: 0.4457452094851268
Weighted F-score: 0.4414301550460955
Accuracy: 0.4413489736070381
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.4564724703372884
Weighted specificity: 0.9564672059777368
Weighted precision: 0.4594303517123525
Weighted F-score: 0.4564912770222197
Accuracy: 0.4564724703372884
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


...

 
Metrics: 
Weighted sensivity: 0.4468
Weighted specificity: 0.9556457596774786
Weighted precision: 0.44927246204566296
Weighted F-score: 0.4464592763884776
Accuracy: 0.4468
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.4449926656887585
Weighted specificity: 0.9554828341564283
Weighted precision: 0.44764723799357425
Weighted F-score: 0.44504520090629157
Accuracy: 0.4449926656887585
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.4481792717086835
Weighted specificity: 0.9555673701415526
Weighted precision: 0.45061519879734235
Weighted F-score: 0.44821446498023626
Accuracy: 0.4481792717086835
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.45577051367578386
Weighted specificity: 0.9564869220602541
Weighted precision: 0.4594854186157434
Weighted F-score: 0.45595261217943556
Accuracy: 0.45577051367578386
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.44915932746196957
Weighted specificity: 0.9557369599963289
Weighted precision: 0.4532850995766088
Weighted F-score: 0.4495912767965312
Accuracy: 0.44915932746196957


In [9]:
##MODEL 4

parameters_knn = Dict("n_neighbors" => 15);

modelCrossValidation(:kNN, parameters_knn, inputs, targets, index)

#Fold 8: F-score 0.39, tiempo inmediato

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.40282403090448915
Weighted specificity: 0.9512084430031544
Weighted precision: 0.4049745520797339
Weighted F-score: 0.40263142722255263
Accuracy: 0.40282403090448915
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.38574283810792803
Weighted specificity: 0.9501007402527051
Weighted precision: 0.38881526161190133
Weighted F-score: 0.38593194250304946
Accuracy: 0.38574283810792803
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.3930951746201013
Weighted specificity: 0.9507741412030924
Weighted precision: 0.3965486645382472
Weighted F-score: 0.3931137035326665
Accuracy: 0.3930951746201013
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.39781362485002
Weighted specificity: 0.9513157501271227
Weighted precision: 0.3998428278656537
Weighted F-score: 0.3974689531994323
Accuracy: 0.39781362485002
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


...

 
Metrics: 
Weighted sensivity: 0.3948
Weighted specificity: 0.9510472515457705
Weighted precision: 0.3985745188374906
Weighted F-score: 0.39509018978572835
Accuracy: 0.3948
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.3919189225230031
Weighted specificity: 0.9503926119978211
Weighted precision: 0.39400710128385696
Weighted F-score: 0.39207953220183056
Accuracy: 0.3919189225230031
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.39442443644124314
Weighted specificity: 0.9507836987483379
Weighted precision: 0.3968044319572754
Weighted F-score: 0.39461359841050175
Accuracy: 0.39442443644124314
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.3949299533022015
Weighted specificity: 0.9510498253676114
Weighted precision: 0.39735062650637076
Weighted F-score: 0.3950765393661975
Accuracy: 0.3949299533022015
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.38777688817720846
Weighted specificity: 0.9500505342966833
Weighted precision: 0.3907534815552779
Weighted F-score: 0.3877907342814144
Accuracy: 0.38777688817720846


Claramente podemos observar que cuanto mayor sea el valor de k menos es el fscore, por lo tanto vamos a probar con k = 1 y k = 2

In [10]:
##MODEL 5

parameters_knn = Dict("n_neighbors" => 2);

modelCrossValidation(:kNN, parameters_knn, inputs, targets, index)

#Fold 8: tiempo inmediato, Fscore: 0.8125

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8083122419075529
Weighted specificity: 0.9854663341193054
Weighted precision: 0.8145029821029163
Weighted F-score: 0.8080895865209939
Accuracy: 0.8083122419075529
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8058627581612259
Weighted specificity: 0.9853736076924019
Weighted precision: 0.8141808842082898
Weighted F-score: 0.8061857067689163
Accuracy: 0.8058627581612259
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.795921087709944
Weighted specificity: 0.9846092494807863
Weighted precision: 0.8038941056840949
Weighted F-score: 0.7964259734534692
Accuracy: 0.795921087709944
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8136248500199973
Weighted specificity: 0.9858892372908332
Weighted precision: 0.8200017480311984
Weighted F-score: 0.8136572368829335
Accuracy: 0.8136248500199973
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


...

 
Metrics: 
Weighted sensivity: 0.8024
Weighted specificity: 0.984968196769385
Weighted precision: 0.8092135640207616
Weighted F-score: 0.8027721956370522
Accuracy: 0.8024
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8073076410188025
Weighted specificity: 0.9853477379878794
Weighted precision: 0.8137506395464774
Weighted F-score: 0.80722514398684
Accuracy: 0.8073076410188025
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8023209283713485
Weighted specificity: 0.9847904299098342
Weighted precision: 0.8104330445215275
Weighted F-score: 0.8025655825797194
Accuracy: 0.8023209283713485
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8126751167444963
Weighted specificity: 0.9859133485936338
Weighted precision: 0.8205024995370092
Weighted F-score: 0.812916667084662
Accuracy: 0.8126751167444963
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8069121964238057
Weighted specificity: 0.9854818557912741
Weighted precision: 0.8137651915385052
Weighted F-score: 0.8070909310718509
Accuracy: 0.8069121964238057


In [11]:
##MODEL 6

parameters_knn = Dict("n_neighbors" => 1);

modelCrossValidation(:kNN, parameters_knn, inputs, targets, index)

#Fold 7: Tiempo inmediato, Fscore: 0.8953

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8913014519781537
Weighted specificity: 0.9911179286304825
Weighted precision: 0.8914525948337454
Weighted F-score: 0.8912493441957999
Accuracy: 0.8913014519781537
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8896735509660226
Weighted specificity: 0.9910571546679154
Weighted precision: 0.889883318844948
Weighted F-score: 0.889675187204717
Accuracy: 0.8896735509660226
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8830978405758464
Weighted specificity: 0.9905232706446396
Weighted precision: 0.8834784964159162
Weighted F-score: 0.8830850785624279
Accuracy: 0.8830978405758464
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.893347553659512
Weighted specificity: 0.9912504427468967
Weighted precision: 0.8934490044656641
Weighted F-score: 0.8932995478816586
Accuracy: 0.893347553659512
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------

...

 
Metrics: 
Weighted sensivity: 0.8837333333333334
Weighted specificity: 0.9905063420305058
Weighted precision: 0.8839592823893451
Weighted F-score: 0.8837548326912179
Accuracy: 0.8837333333333334
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8947859714628618
Weighted specificity: 0.9913584724072789
Weighted precision: 0.8950685888187103
Weighted F-score: 0.8947869963715892
Accuracy: 0.8947859714628618
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8831532613045218
Weighted specificity: 0.9903373757101384
Weighted precision: 0.8837509502029072
Weighted F-score: 0.8831957503293356
Accuracy: 0.8831532613045218
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8939292861907938
Weighted specificity: 0.9913601044757324
Weighted precision: 0.8941999985039184
Weighted F-score: 0.8939214399857962
Accuracy: 0.8939292861907938
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------


------------------------------------------------------------------------------------------------------------------
 
Metrics: 
Weighted sensivity: 0.8905791299706431
Weighted specificity: 0.9912130595172782
Weighted precision: 0.8907240947285614
Weighted F-score: 0.8905715408801587
Accuracy: 0.8905791299706431


In [48]:
#Como abrir los modelos (los outputs pq el modelo da error)

folder_path = "modelknn 3"

# File path to load the model from inside the folder
model_file_path = joinpath(folder_path, "targets_knnmodel1.jld2")

targets_model = nothing

# Load the model from the JLD file
loaded_model = JLD2.jldopen(model_file_path) do file
     global targets_model
     targets_model = read(file, "targets_test")
end

model2_file_path = joinpath(folder_path, "outputs_knnmodel1.jld2")

outputs_model = nothing

# Load the model from the JLD file
loaded_model = JLD2.jldopen(model2_file_path) do file
     global outputs_model
     outputs_model = read(file, "outputs_model")
end

confusionMatrix(outputs_model, targets_model)


([1.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 2.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], 0.12962962962962962, 0.9387847079413819, 0.1503478411566647, 0.9301108315997617, 0.12583955909549305, 0.11254449254449253, 0.9378942891714009, 0.11528562940327647, 0.937931561851277, 0.10345042470429468, 0.8839506172839506, 0.11604938271604937)

### Decision Trees (6 modelos diferentes profundidades)

In [51]:
include("functions_finalproject.jl")

trainClassEnsemble (generic function with 1 method)

In [12]:
##MODEL 1

parameters_dt = Dict("max_depth" => 5, "random_state" => 1);

modelCrossValidation(:DecisionTree, parameters_dt, inputs, targets, index)

#Inmediato, cualquier fold con F-score<0.1

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------

 
Metrics: 
Weighted sensivity: 0.1180231783668576
Weighted specificity: 0.9062595599556816
Weighted precision: 0.10602141264413399
Weighted F-score: 0.07808112923005592
Accuracy: 0.1180231783668576
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.12111925383077948
Weighted specificity: 0.9095504782249013
Weighted precision: 0.13361971869769998
Weighted F-score: 0.08902203468331761
Accuracy: 0.12111925383077948
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.11943481738203146
Weighted specificity: 0.9084878830904489
Weighted precision: 0.15063912127654955
Weighted F-score: 0.08256020306196071
Accuracy: 0.11943481738203146
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.12584988668177577
Weighted specificity: 0.9069837661320305
Weighted precision: 0.16683965342195717
Weighted F-score: 0.08156758821871461
Accuracy: 0.12584988668177577
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------

...

 
Metrics: 
Weighted sensivity: 0.11933333333333333
Weighted specificity: 0.9062044113982352
Weighted precision: 0.09760293515267504
Weighted F-score: 0.08420386679440868
Accuracy: 0.11933333333333333
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.12401653553807174
Weighted specificity: 0.9080648442086207
Weighted precision: 0.10884149873624648
Weighted F-score: 0.0895225927857277
Accuracy: 0.12401653553807174
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.1237828464719221
Weighted specificity: 0.9089834737285413
Weighted precision: 0.15799378577974074
Weighted F-score: 0.08443223098812865
Accuracy: 0.1237828464719221
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.12354903268845897
Weighted specificity: 0.9063596944132355
Weighted precision: 0.13405332961442684
Weighted F-score: 0.08520440149188112
Accuracy: 0.12354903268845897
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.12583400053376034
Weighted specificity: 0.9086462619505608
Weighted precision: 0.11757343098624852
Weighted F-score: 0.08748350626606931
Accuracy: 0.12583400053376034


In [13]:
##MODEL 2

parameters_dt = Dict("max_depth" => 20, "random_state" => 1);

modelCrossValidation(:DecisionTree, parameters_dt, inputs, targets, index)

#Fold 8 : Tiempo 1-2 min, f-score: 0.62

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.5588117756760357
Weighted specificity: 0.9611885060421936
Weighted precision: 0.5732803895155415
Weighted F-score: 0.560496681836396
Accuracy: 0.5588117756760357
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.5726848767488341
Weighted specificity: 0.9621313369462363
Weighted precision: 0.5859267290315655
Weighted F-score: 0.5736624429055643
Accuracy: 0.5726848767488341
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.5539856038389763
Weighted specificity: 0.9627631519967287
Weighted precision: 0.5590586029995326
Weighted F-score: 0.5542298608342673
Accuracy: 0.5539856038389763
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.5539261431809092
Weighted specificity: 0.9611448395285936
Weighted precision: 0.568119653643795
Weighted F-score: 0.554437273465316
Accuracy: 0.5539261431809092
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------

...

-------------------------------------------------- FOLD: 6 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------
 
Metrics: 
Weighted sensivity: 0.5830666666666666
Weighted specificity: 0.9649401135031184
Weighted precision: 0.5970709185709374
Weighted F-score: 0.5843085239642822
Accuracy: 0.5830666666666666
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.5666088811841579
Weighted specificity: 0.9635985284425922
Weighted precision: 0.5764236550559977
Weighted F-score: 0.5671063195531529
Accuracy: 0.5666088811841579
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.58796852074163
Weighted specificity: 0.9654797719752228
Weighted precision: 0.5991553790648809
Weighted F-score: 0.5899846953615965
Accuracy: 0.58796852074163
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.5533022014676451
Weighted specificity: 0.9629405517984316
Weighted precision: 0.5617165809017716
Weighted F-score: 0.553758428084975
Accuracy: 0.5533022014676451
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.5641846810781959
Weighted specificity: 0.962225039927877
Weighted precision: 0.5810084550932865
Weighted F-score: 0.5663897898131172
Accuracy: 0.5641846810781959


In [14]:
##MODEL 3

parameters_dt = Dict("max_depth" => 50, "random_state" => 1);

modelCrossValidation(:DecisionTree, parameters_dt, inputs, targets, index)

#Fold 8: tiempo 1-2 min, f-score: 0.875

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8725189822832023
Weighted specificity: 0.989504117674875
Weighted precision: 0.8727545540060898
Weighted F-score: 0.8724725246033049
Accuracy: 0.8725189822832023
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8706195869420387
Weighted specificity: 0.9893755172265103
Weighted precision: 0.8708677460796386
Weighted F-score: 0.870618004933345
Accuracy: 0.8706195869420387
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8675019994668088
Weighted specificity: 0.9890604263023425
Weighted precision: 0.867795077349217
Weighted F-score: 0.8674996931707731
Accuracy: 0.8675019994668088
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8725503266231169
Weighted specificity: 0.989595985499961
Weighted precision: 0.8728212794270879
Weighted F-score: 0.8725665327582136
Accuracy: 0.8725503266231169
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


...


 
Metrics: 
Weighted sensivity: 0.8653333333333333
Weighted specificity: 0.9889773553571104
Weighted precision: 0.8653825351579709
Weighted F-score: 0.865300305288304
Accuracy: 0.8653333333333333
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8733164421922923
Weighted specificity: 0.9893619153384083
Weighted precision: 0.8736272018061924
Weighted F-score: 0.8732656516122579
Accuracy: 0.8733164421922923
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8647458983593438
Weighted specificity: 0.9888891461837117
Weighted precision: 0.865029725035095
Weighted F-score: 0.8647289669586778
Accuracy: 0.8647458983593438
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8737825216811207
Weighted specificity: 0.9896132876812728
Weighted precision: 0.8740970763190727
Weighted F-score: 0.8738342692251049
Accuracy: 0.8737825216811207
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8709634374166
Weighted specificity: 0.989265399937045
Weighted precision: 0.8711088951329221
Weighted F-score: 0.8709073425631794
Accuracy: 0.8709634374166


In [15]:
##MODEL 4

parameters_dt = Dict("max_depth" => 100, "random_state" => 1);

modelCrossValidation(:DecisionTree, parameters_dt, inputs, targets, index)

#Fold : Tiempo 1-2 min, F-score: 0.875

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8725189822832023
Weighted specificity: 0.989504117674875
Weighted precision: 0.8727545540060898
Weighted F-score: 0.8724725246033049
Accuracy: 0.8725189822832023
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8706195869420387
Weighted specificity: 0.9893755172265103
Weighted precision: 0.8708677460796386
Weighted F-score: 0.870618004933345
Accuracy: 0.8706195869420387
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------


------------------------------------------------------------------------------------------------------------------
 
Metrics: 
Weighted sensivity: 0.8680351906158358
Weighted specificity: 0.9890413490592279
Weighted precision: 0.8682426462564926
Weighted F-score: 0.8680012430945722
Accuracy: 0.8680351906158358
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8725503266231169
Weighted specificity: 0.989595985499961
Weighted precision: 0.8728212794270879
Weighted F-score: 0.8725665327582136
Accuracy: 0.8725503266231169
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------

...

 
Metrics: 
Weighted sensivity: 0.8653333333333333
Weighted specificity: 0.9889773553571104
Weighted precision: 0.8653825351579709
Weighted F-score: 0.865300305288304
Accuracy: 0.8653333333333333
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8733164421922923
Weighted specificity: 0.9893619153384083
Weighted precision: 0.8736272018061924
Weighted F-score: 0.8732656516122579
Accuracy: 0.8733164421922923
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8647458983593438
Weighted specificity: 0.9888891461837117
Weighted precision: 0.865029725035095
Weighted F-score: 0.8647289669586778
Accuracy: 0.8647458983593438
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8737825216811207
Weighted specificity: 0.9896132876812728
Weighted precision: 0.8740970763190727
Weighted F-score: 0.8738342692251049
Accuracy: 0.8737825216811207
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8698959167333867
Weighted specificity: 0.9892260357794278
Weighted precision: 0.8699895792400253
Weighted F-score: 0.8698475367635538
Accuracy: 0.8698959167333867


In [16]:
##MODEL 5

parameters_dt = Dict("max_depth" => 40, "random_state" => 1);

modelCrossValidation(:DecisionTree, parameters_dt, inputs, targets, index)

#Fold 8: Tiempo 1-2 min, F-score: 0.874

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8678566671106966
Weighted specificity: 0.9892051026255978
Weighted precision: 0.8679711009227422
Weighted F-score: 0.8677837341144881
Accuracy: 0.8678566671106966
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8663557628247834
Weighted specificity: 0.9891572718661824
Weighted precision: 0.8665734676135721
Weighted F-score: 0.8663530990477606
Accuracy: 0.8663557628247834
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8624366835510531
Weighted specificity: 0.988734993151925
Weighted precision: 0.8625350966886839
Weighted F-score: 0.8623570896434081
Accuracy: 0.8624366835510531
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8702839621383816
Weighted specificity: 0.9895238020384078
Weighted precision: 0.8704971972948673
Weighted F-score: 0.8702884108634712
Accuracy: 0.8702839621383816
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


...


 
Metrics: 
Weighted sensivity: 0.8650666666666667
Weighted specificity: 0.9889601845402493
Weighted precision: 0.865042809360752
Weighted F-score: 0.8649978446007014
Accuracy: 0.8650666666666667
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8713161754900653
Weighted specificity: 0.989331374988748
Weighted precision: 0.8714163547113647
Weighted F-score: 0.8712602547604763
Accuracy: 0.8713161754900653
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8632786447912498
Weighted specificity: 0.9888393997996423
Weighted precision: 0.8635077872569882
Weighted F-score: 0.863262224076503
Accuracy: 0.8632786447912498
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8740493662441627
Weighted specificity: 0.9898419522525731
Weighted precision: 0.8744660071547131
Weighted F-score: 0.8741046586151371
Accuracy: 0.8740493662441627
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8656258340005337
Weighted specificity: 0.9889446448746039
Weighted precision: 0.865887611518176
Weighted F-score: 0.8655908100919506
Accuracy: 0.8656258340005337


In [17]:
##MODEL 6

parameters_dt = Dict("max_depth" => 30, "random_state" => 1);

modelCrossValidation(:DecisionTree, parameters_dt, inputs, targets, index)

#Fold 8: Tiempo 1-2 min, F-score: 0.842

 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 1 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8096443319568403
Weighted specificity: 0.984386282278957
Weighted precision: 0.8105375860048398
Weighted F-score: 0.8097346579607162
Accuracy: 0.8096443319568403
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 2 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8154563624250499
Weighted specificity: 0.984703553599974
Weighted precision: 0.8169291969156448
Weighted F-score: 0.8155528529524467
Accuracy: 0.8154563624250499
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 3 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8261796854172221
Weighted specificity: 0.9854338844235112
Weighted precision: 0.8271984651812826
Weighted F-score: 0.8262316746572361
Accuracy: 0.8261796854172221
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 4 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8305559258765498
Weighted specificity: 0.9865211250498355
Weighted precision: 0.8314144278233432
Weighted F-score: 0.8306486383820569
Accuracy: 0.8305559258765498
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 5 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------

...


 
Metrics: 
Weighted sensivity: 0.8398666666666667
Weighted specificity: 0.9870965566405642
Weighted precision: 0.8400496238386062
Weighted F-score: 0.8398874416353911
Accuracy: 0.8398666666666667
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 7 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8337111614881985
Weighted specificity: 0.986718806349077
Weighted precision: 0.834904146898819
Weighted F-score: 0.8338633028803389
Accuracy: 0.8337111614881985
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 8 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8225957049486461
Weighted specificity: 0.9851205620290491
Weighted precision: 0.8239226705870787
Weighted F-score: 0.8227426771653313
Accuracy: 0.8225957049486461
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 9 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.8392261507671781
Weighted specificity: 0.9872717261974869
Weighted precision: 0.8404737472751049
Weighted F-score: 0.839568230769047
Accuracy: 0.8392261507671781
 
------------------------------------------------------------------------------------------------------------------
-------------------------------------------------- FOLD: 10 -------------------------------------------------------
------------------------------------------------------------------------------------------------------------------


 
Metrics: 
Weighted sensivity: 0.828796370429677
Weighted specificity: 0.9857786636299714
Weighted precision: 0.830100138920832
Weighted F-score: 0.8288695207803307
Accuracy: 0.828796370429677


In [56]:
#Como abrir los modelos (los outputs pq el modelo da error)

folder_path = "modeldt 5"

# File path to load the model from inside the folder
model_file_path = joinpath(folder_path, "targets_dtmodel1.jld2")

targets_model = nothing

# Load the model from the JLD file
loaded_model = JLD2.jldopen(model_file_path) do file
     global targets_model
     targets_model = read(file, "targets_test")
end

model2_file_path = joinpath(folder_path, "outputs_dtmodel1.jld2")

outputs_model = nothing

# Load the model from the JLD file
loaded_model = JLD2.jldopen(model2_file_path) do file
     global outputs_model
     outputs_model = read(file, "outputs_model")
end

confusionMatrix(outputs_model, targets_model)


([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], 0.14814814814814814, 0.9035574606808254, 0.049457994579945805, 0.9348584167430324, 0.06634806849860614, 0.09393939393939395, 0.9367803739219317, 0.03035230352303523, 0.9400832289295309, 0.04100358422939068, 0.8864197530864197, 0.11358024691358026)

Al aumentar la profundidad conseguimos resultados mejores, llegando a un punto donde se estanca (usa una determinada profundidad simplemente) en nuestro caso a partir de una profundidad de 30 los resultados que se consiguen son simmilares (mejores con 40). Así, pese a que computacionalmente en este caso no es muy caro, el mejor modelo es el de profundidad 40 consiguiendo resultados similares a los de profundidades mayores pero al tener menor profundidad con un coste computacional menor. 

### Ensemble models

Viendo los modelos claramente conseguimos los mejores resultados en las SVM, kNN y decision trees. Probaremos a hacer un ensemble model con todos ellos. 

In [29]:
include("functions_finalproject.jl")
parameters = [
    Dict("model" => "kNN", "n_neighbors" => 1);
    Dict("model" => "DecisionTree", "max_depth" => 30, "random_state" => 1);
    Dict("model" => "DecisionTree", "max_depth" => 30, "random_state" => 1);
    ];

parameters = Vector{Dict}(parameters);
estimators = [:SVM, :kNN,];

In [30]:
@sk_import neighbors: KNeighborsClassifier;
@sk_import ensemble:(AdaBoostClassifier, GradientBoostingClassifier)
@sk_import svm:SVC
@sk_import ensemble:BaggingClassifier
@sk_import ensemble:StackingClassifier
@sk_import ensemble:VotingClassifier
@sk_import svm:SVC
@sk_import tree:DecisionTreeClassifier
@sk_import linear_model:LogisticRegression
@sk_import naive_bayes:GaussianNB 

PyObject <class 'sklearn.naive_bayes.GaussianNB'>

In [31]:
trainClassEnsemble(estimators, parameters, (inputs, targets), index);

#Ordenador cristian 53 mins. Accuracy 88% (pilla mal la media pq usa un vector vacio y al hacer un fold solo lo calcula mal) consigue un f1-score de 0.8859

In [60]:
#Como abrir los modelos (los outputs pq el modelo da error)

folder_path = "modelensemble"

# File path to load the model from inside the folder
model_file_path = joinpath(folder_path, "targets_ensemblemodel1.jld2")

targets_model = nothing

# Load the model from the JLD file
loaded_model = JLD2.jldopen(model_file_path) do file
     global targets_model
     targets_model = read(file, "targets_test")
end

model2_file_path = joinpath(folder_path, "outputs_ensemblemodel1.jld2")

outputs_model = nothing

# Load the model from the JLD file
loaded_model = JLD2.jldopen(model2_file_path) do file
     global outputs_model
     outputs_model = read(file, "outputs_model")
end

confusionMatrix(outputs_model, targets_model)


([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], 0.14814814814814814, 0.9035574606808254, 0.049457994579945805, 0.9348584167430324, 0.06634806849860614, 0.09393939393939395, 0.9367803739219317, 0.03035230352303523, 0.9400832289295309, 0.04100358422939068, 0.8864197530864197, 0.11358024691358026)